# DB2-Salesforce connector: pull in new email flags from DB2

In [1]:
# Parameters
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')

salesforce = application.new_salesforce_engine()
db_s = salesforce

nanoHUB - Serving Students, Researchers & Instructors


2021-08-26 00:40:19,558| INFO    | 0 keys loaded from agent


2021-08-26 00:40:19,558 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_agent_keys:1060]: 0 keys loaded from agent


2021-08-26 00:40:19,561| INFO    | 0 key(s) loaded


2021-08-26 00:40:19,561 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_keys:1117]: 0 key(s) loaded


...

2021-08-26 00:40:19,562 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.__init__:978]: Connecting to gateway: db2.nanohub.org:22 as user 'saxenap'


2021-08-26 00:40:19,564 - [INFO] nanoHUB.containers.dataaccess [connection.get_connection_for:101]: Started SSH Tunnel with db2.nanohub.org


2021-08-26 00:40:19,880| INFO    | Opening tunnel: 0.0.0.0:54269 <> 127.0.0.1:3306


2021-08-26 00:40:19,880 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel._serve_forever_wrapper:1433]: Opening tunnel: 0.0.0.0:54269 <> 127.0.0.1:3306


Obtained Salesforce access token ...... True


In [2]:
import pandas as pd
import datetime

## Get the latest email opt in/out data from DB2


In [3]:
# get new preferences for users
pcomms = pd.read_sql_query("select id, user_id, profile_key, profile_value from jos_user_profiles \
where profile_key in ('personalizedcommunication');",nanohub_db)#order by rand() limit 50000;",engine) #and modifiedDate >= '2019-08-08'

up_news = pd.read_sql_query("select id, user_id, profile_key, profile_value from jos_user_profiles \
where profile_key in ('updates_news');",nanohub_db) #order by rand() limit 50000;",engine)

In [4]:
display(pcomms.head(2))
display(pcomms.tail(2))
display(up_news.head(2))
display(up_news.tail(2))

,id,user_id,profile_key,profile_value
0,1524493,998,personalizedcommunication,"Yes, I would like to receive personalized comm..."
1,1524495,1000,personalizedcommunication,"Yes, I would like to receive personalized comm..."


,id,user_id,profile_key,profile_value
244530,2173256,336333,personalizedcommunication,"No, I do not want to receive personalized comm..."
244531,2173263,335734,personalizedcommunication,"Yes, I would like to receive personalized comm..."


,id,user_id,profile_key,profile_value
0,1524494,998,updates_news,"No, I do not want to receive updates and news"
1,1524496,1000,updates_news,"No, I do not want to receive updates and news"


,id,user_id,profile_key,profile_value
244486,2173257,336333,updates_news,"No, I do not want to receive updates and news"
244487,2173264,335734,updates_news,"Yes, I would like to receive updates and news"


In [5]:
print(pcomms.shape)
print(up_news.shape)

(244532, 4)
(244488, 4)


In [6]:
## adjusting new_email dataframe for the new email optin values 
# 0 or 1
pcomms_t = pcomms[['user_id','profile_value']]
up_news_t = up_news[['user_id','profile_value']]

display(pcomms_t.head(2))
display(up_news_t.head(2))

,user_id,profile_value
0,998,"Yes, I would like to receive personalized comm..."
1,1000,"Yes, I would like to receive personalized comm..."


,user_id,profile_value
0,998,"No, I do not want to receive updates and news"
1,1000,"No, I do not want to receive updates and news"


In [7]:
## filtering pcomms_t with 0 or 1
ptt = pcomms_t['profile_value'].to_list()

for index, val in enumerate(ptt):
    val = val.split(',')[0]
    if val == 'Yes':
        ptt[index] = 1 # 1 indicates opt-in
    else:
        ptt[index] = 0 # 1 indicates opt-out

pcomms_t.loc[:,'profile_value'] = ptt


2021-08-26 00:40:34,528| WARNING | /Users/saxenap/.virtualenvs/nanohub/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)



2021-08-26 00:40:34,528| WARNING | /Users/saxenap/.virtualenvs/nanohub/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)



2021-08-26 00:40:34,528 - [WARNING] py.warnings [warnings._showwarnmsg:109]: /Users/saxenap/.virtualenvs/nanohub/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)



In [8]:
## filtering up news t with 0 or 1
untt = up_news_t['profile_value'].to_list()

for index, val in enumerate(untt):
    val = val.split(',')[0]
    if val == 'Yes':
        untt[index] = 1 # 1 indicates opt-in
    else:
        untt[index] = 0 # 1 indicates opt-out

up_news_t.loc[:,'profile_value'] = untt


In [9]:
## display
display(pcomms_t.head(2))
display(pcomms_t.tail(2))
display(up_news_t.head(2))
display(up_news_t.tail(2))

,user_id,profile_value
0,998,1
1,1000,1


,user_id,profile_value
244530,336333,0
244531,335734,1


,user_id,profile_value
0,998,0
1,1000,0


,user_id,profile_value
244486,336333,0
244487,335734,1


In [10]:
## create new dataframe
new_opts = pd.DataFrame()

new_opts['nanoHUB_user_ID__c'] = pcomms_t['user_id']
new_opts['personalizedcommunication__c'] = pcomms_t['profile_value']
new_opts['updatesnews__c'] = up_news_t['profile_value']


In [11]:
display(new_opts.head(2))
print(type(new_opts))

,nanoHUB_user_ID__c,personalizedcommunication__c,updatesnews__c
0,998,1,0.0
1,1000,1,0.0


<class 'pandas.core.frame.DataFrame'>


## Send to SF

In [12]:
db_s.object_id = object_id
db_s.external_id = external_id

# send data to Salesforce
db_s.send_data(new_opts)


[Success] Bulk job creation successful. Job ID = 7505w00000ZJL5tAAH
hello


[Success] CSV upload successful. Job ID = 7505w00000ZJL5tAAH


[Success] Closing job successful. Job ID = 7505w00000ZJL5tAAH


In [13]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000ZJL5tAAH',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-08-26T04:40:35.000+0000',
 'systemModstamp': '2021-08-26T04:40:41.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [14]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''
